In [6]:
import airflow
from airflow import DAG
from airflow.operators.python import PythonOperator

from datetime import datetime, timedelta  

import scrappingBooks
import processingBooks
import main

import warnings
warnings.filterwarnings("ignore")

In [7]:
#DAG setup
default_args = {
    'owner': 'airflow',
    'depends_on_past': False,
    # Example: Initiate at may/20/2022
    'start_date': datetime(2022, 5, 20),
    'email': ['lucas1613@gmail.com'],
    'email_on_failure': False,
    'email_on_retry': False,
    # In case of errors, try to run again just 1 time
    'retries': 1,
    # Try again after 30 seconds after the error
    'retry_delay': timedelta(seconds=30),
    # Run once every 6 hours
    'schedule_interval': '*/2 * * * *' #### change accordly desire interval
}

with DAG(dag_id='updateBooks', 
        default_args=default_args, 
        schedule_interval=None,
        tags=['currency']
        ) as dag:
    
    
    #tasks to scrappe new data from web source
    t1 = PythonOperator(
        task_id='scrappingBooks',
        python_callable= scrappingBooks.getLinksPages(),
        dag=dag,
    )
    
    #ler a tabela html no site
    t2 = PythonOperator(
        task_id='checkDatabase',
        python_callable= processingBooks.createDatabase(),
        dag=dag,
    )

    #aplicar transformações na tabela
    t3 = PythonOperator(
        task_id='transformUpdateData',
        python_callable= processingBooks.insertData(),
        dag=dag,
    )
    
    #salvar o arquivo csv
    t4 = PythonOperator(
        task_id='getCopiesBooks',
        python_callable= main.amountCopiesBooks(),
        dag=dag,
    )
    
    #dependências entre as tarefas
    t1 >> t2 >> t3 >> t4

Scrapping from https://books.toscrape.com/ ...


AirflowException: `python_callable` param must be callable

ModuleNotFoundError: No module named 'pwd'